In [150]:
import random
import os
import threading
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, wait

from tqdm.notebook import tqdm, trange
from pathlib import Path
from random import randint
from ui.python.Layout import Layout
import numpy as np
import plotly.express as px

In [151]:
import pandas as pd
from pandas.core.common import flatten

In [152]:
layout_path = './../data/layout 12x15_1.json'
MAX_WORKERS = 24
USE_CATEGORY_FRO_CHECKS = True

In [153]:
selected_categories = [
 'bakery',
 'beverages',
 'breakfast',
 'canned goods',
 'dairy eggs',
 'deli',
 'dry goods pasta',
 'frozen',
 'household',
 'meat seafood',
 'pantry',
 'produce',
 'snacks']

# Preprocessing

In [154]:
df = pd.read_csv('./../data/datasets/ECommerce_consumer behaviour.csv')
df

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id,department,product_name
0,2425083,49125,1,2,18,NaN,17,1,0,13,pantry,baking ingredients
1,2425083,49125,1,2,18,NaN,91,2,0,16,dairy eggs,soy lactosefree
2,2425083,49125,1,2,18,NaN,36,3,0,16,dairy eggs,butter
3,2425083,49125,1,2,18,NaN,83,4,0,4,produce,fresh vegetables
4,2425083,49125,1,2,18,NaN,83,5,0,4,produce,fresh vegetables
...,...,...,...,...,...,...,...,...,...,...,...,...
2019496,3390742,199430,16,3,18,5.0,83,8,0,4,produce,fresh vegetables
2019497,458285,128787,42,2,19,3.0,115,1,1,7,beverages,water seltzer sparkling water
2019498,458285,128787,42,2,19,3.0,32,2,1,4,produce,packaged produce
2019499,458285,128787,42,2,19,3.0,32,3,1,4,produce,packaged produce


In [155]:
df = df[['order_id', 'user_id', 'order_number', 'product_id', 'product_name', 'department', 'department_id']]
df

,order_id,user_id,order_number,product_id,product_name,department,department_id
0,2425083,49125,1,17,baking ingredients,pantry,13
1,2425083,49125,1,91,soy lactosefree,dairy eggs,16
2,2425083,49125,1,36,butter,dairy eggs,16
3,2425083,49125,1,83,fresh vegetables,produce,4
4,2425083,49125,1,83,fresh vegetables,produce,4
...,...,...,...,...,...,...,...
2019496,3390742,199430,16,83,fresh vegetables,produce,4
2019497,458285,128787,42,115,water seltzer sparkling water,beverages,7
2019498,458285,128787,42,32,packaged produce,produce,4
2019499,458285,128787,42,32,packaged produce,produce,4


In [156]:
df['product_name'].unique()

array(['baking ingredients', 'soy lactosefree', 'butter',
       'fresh vegetables', 'yogurt', 'canned meals beans',
       'poultry counter', 'ice cream ice', 'fresh fruits', 'milk',
       'packaged cheese', 'bread', 'tea', 'bakery desserts',
       'frozen breakfast', 'cereal', 'eggs', 'buns rolls', 'cream',
       'water seltzer sparkling water', 'pickled goods olives',
       'packaged poultry', 'other creams cheeses',
       'honeys syrups nectars', 'coffee', 'refrigerated',
       'energy granola bars', 'soft drinks', 'latino foods',
       'plates bowls cups flatware', 'paper goods', 'oral hygiene',
       'diapers wipes', 'food storage', 'nuts seeds dried fruit', 'soap',
       'packaged vegetables fruits', 'hot dogs bacon sausage',
       'lunch meat', 'chips pretzels', 'meat counter',
       'fresh dips tapenades', 'prepared soups salads', 'condiments',
       'juice nectars', 'canned fruit applesauce',
       'preserved dips spreads', 'packaged produce',
       'canned jarr

In [157]:
# return tuple {orderId, list(items)} for single check
def get_order_items(order_id):
    order = df[df['order_id'] == order_id]
    if not USE_CATEGORY_FRO_CHECKS:
        return order_id, order['product_name'].unique().tolist()
    else:
        is_in_category = order['department'].apply(lambda x: x in selected_categories)
        return order_id, order[is_in_category]['product_name'].unique().tolist()

In [158]:
if USE_CATEGORY_FRO_CHECKS:
    df = df[df['department'].isin(selected_categories)]

In [159]:
check_ids = df['order_id'].unique().tolist()
check_list = []
for check_id in tqdm(check_ids[:10000]):
    check = get_order_items(check_id)
    check_list.append(check)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [160]:
len(check_list)

10000

In [161]:
if USE_CATEGORY_FRO_CHECKS:
    items_list = df[df['department'].apply(lambda x: x in selected_categories)]['product_name'].unique().tolist()
else:
    items_list = df['product_name'].unique().tolist()

In [162]:
len(items_list)

98

# Layout

In [163]:
layout_test = Layout(layout_path).get_empty_rack_layout()
#curr_dir = Path(os.getcwd()).parent
#layout_test.display_in_window(home_dir=str(curr_dir))

In [164]:
def random_layout():
    layout = Layout(layout_path)
    layout.set_item_list(items_list)
    for row in range(layout.shape[0]):
        for col in range(layout.shape[1]):
            if layout[row][col].type.name == 'RACK':
                for lev in range(layout.get_max_rack_level()):
                    layout.set_item_to_rack(random.choice(layout.get_item_list()), (row, col), level=lev)
    return layout

In [165]:
def layout_with_one_item(max_rack_level=4):
    layout = Layout(layout_path)
    layout.set_item_list(items_list)
    layout.set_max_rack_level(max_rack_level)
    for row in range(layout.shape[0]):
        for col in range(layout.shape[1]):
            if layout[row][col].type.name == 'RACK':
                for lev in range(layout.get_max_rack_level()):
                    layout.set_item_to_rack(layout.get_item_list()[0], (row, col), level=lev)
    return layout

In [166]:
layouts = []
for i in range(10):
    layouts.append(random_layout())

In [167]:
# dir = Path(os.getcwd()).parent
# layouts[2].display_in_window(home_dir=str(dir))

In [168]:
layouts[3]

# Evaluate

In [169]:
from helpers.estimation_helpers import evaluate_layout, thread_func, calculate_score

In [170]:
for layout in layouts:
    print(evaluate_layout(layout, check_list[0][1], use_item_count=True))

(62, False)
(0, True, ['baking ingredients', 'soy lactosefree', 'yogurt'])
(61, False)
(0, True, ['soy lactosefree', 'yogurt', 'poultry counter'])
(49, False)
(0, True, ['butter'])
(0, True, ['yogurt'])
(0, True, ['butter'])
(0, True, ['soy lactosefree'])
(49, False)


In [171]:
#dirr = Path(os.getcwd()).parent
#layouts[2].display_in_window(home_dir=str(dirr))

# Mutate layout

In [172]:
def select_n_best_layouts(layouts, checks, n_best=2, start_pos=None, reward_type='max', debug=False, use_item_count=False, weights=(500, 150, 350)):
    res = dict()
    f_res = dict()
    
    # shuffle checks
    #random.shuffle(checks)
    
    def format_debug_string(score):
        return f'Path: {score["path"]}, Invalid: {score["invalid"]}, Uniformity: {score["rack_uniformity"]},{score["tile_uniformity"]}'
    
    def format_short_debug_string(score):
        return f'P:{score["path"]}, I:{score["invalid"]}, U:{score["rack_uniformity"]},{score["tile_uniformity"]}'
    
    def optimal_score(score):
        return score['invalid'] * 10000 + score['rack_uniformity'] * 100 + score['tile_uniformity']
    
    if MAX_WORKERS > 1:
        futures = list()
        with ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
            for layout in layouts:
                futures.append(executor.submit(thread_func, layout, checks, start_pos, use_item_count))
        
        #done, not_done = wait(futures, return_when='ALL_COMPLETED')
        for future in futures:
            # store modified layout as its path count is changed
            score, n_layout = future.result()
            res[n_layout] = score
    else:
        for layout in layouts:
            score, n_layout = thread_func(layout, checks, start_pos, use_item_count)
            res[n_layout] = score
    f_res = dict()
    if reward_type == 'max':
        res = sorted(res.items(), key=lambda x: x[1]['path'], reverse=True)
    elif reward_type == 'min':
        for key in res.keys():
            res[key]['path'] = res[key]['path'] + res[key]['invalid']*1000
        res = sorted(res.items(), key=lambda x: x[1]['path'])
    elif reward_type == 'valid':
        res = sorted(res.items(), key=lambda x: x[1]['invalid'])
    elif reward_type == 'uniformity':
        res = sorted(res.items(), key=lambda x: optimal_score(x[1]))
    elif reward_type == 'score':
        for key in res.keys():
            f_res[key] = calculate_score(res[key], key, checks, weights)
        f_res = sorted(f_res.items(), key=lambda x: x[1], reverse=False)

    if debug:
        if reward_type == 'score':
            print(f"Scores: ", end='')
            print([f[1] for f in f_res[:n_best]])
            return [x[0] for x in f_res[:n_best]], [x[1] for x in f_res[:n_best]], [res[x[0]]['missing_items'] for x in f_res[:n_best]]
        print([format_short_debug_string(x[1]) for x in res[:n_best]])
    return [x[0] for x in res[:n_best]], [x[1] for x in res[:n_best]], [x[1]['missing_items'] for x in res[:n_best]]

In [173]:
#MAX_WORKERS = 1
new_layouts, hi, _ = select_n_best_layouts(layouts, check_list[:1], debug=True, reward_type='uniformity', use_item_count=True)

['P:62, I:0, U:360,210', 'P:49, I:0, U:362,208']


In [174]:
new_layouts

In [175]:
def mutate_layout(original_layout, alpha=0.01):
    new_layout = original_layout.copy()
    new_layout.reset_path_count()
    for row in range(new_layout.shape[0]):
        for col in range(new_layout.shape[1]):
            if new_layout[row][col].type.name == 'RACK':
                for lev in range(new_layout.get_max_rack_level()):
                    if random.random() < alpha:
                        new_layout.set_item_to_rack(random.choice(new_layout.get_item_list()), (row, col), level=lev)
    return new_layout

In [176]:
def mutate_uniformity(original_layout, alpha=0.01):
    def get_neighbours(layout, i, j, level):
        floor_pos = None
        left_item = None
        right_item = None
        for pos in [(i, j-1), (i, j+1), (i-1, j), (i+1, j)]:
            if 0 <= pos[0] < layout.shape[0] and 0 <= pos[1] < layout.shape[1]:
                if layout[pos[0]][pos[1]].type.name == 'FLOOR':
                    floor_pos = pos
                    break
        diff = (floor_pos[0] - i, floor_pos[1] - j)
        pos_left = (i - diff[1], j - diff[0])
        pos_right = (i + diff[1], j + diff[0])
        if layout[pos_left[0]][pos_left[1]].type.name == 'RACK':
            left_item = layout[pos_left[0]][pos_left[1]].items[level][0]
        if layout[pos_right[0]][pos_right[1]].type.name == 'RACK':
            right_item = layout[pos_right[0]][pos_right[1]].items[level][0]
        return left_item, right_item
    
    new_layout = original_layout.copy()
    new_layout.reset_path_count()
    for row in range(new_layout.shape[0]):
        for col in range(new_layout.shape[1]):
            if new_layout[row][col].type.name == 'RACK':
                for lev in range(new_layout.get_max_rack_level()):
                    if random.random() < alpha:
                        left_item, right_item = get_neighbours(new_layout, row, col, lev)
                        # if both neighbours are not None
                        if left_item and right_item:
                            # if both neighbours are the same - set the same item to the current rack
                            if left_item == right_item:
                                new_layout.set_item_to_rack(left_item, (row, col), level=lev)
                            # if neighbours are different - set randomly one of them
                            else:
                                new_layout.set_item_to_rack(random.choice([left_item, right_item]), (row, col), level=lev)
                        # if one of the neighbours is None - set existing item to the current rack
                        elif left_item:
                            new_layout.set_item_to_rack(left_item, (row, col), level=lev)
                        elif right_item:
                            new_layout.set_item_to_rack(right_item, (row, col), level=lev)
    return new_layout

In [177]:
def mutate_missing_items(original_layout, missing_items_array, alpha=0.01):
    new_layout = original_layout.copy()
    new_layout.reset_path_count()
    
    def get_present_and_missing_items(layout):
        present_items = dict()
        missing_items = []
        for row in range(layout.shape[0]):
            for col in range(layout.shape[1]):
                if layout[row][col].type.name == 'RACK':
                    for lev in range(layout.get_max_rack_level()):
                        if layout[row][col].items[lev][0] in present_items.keys():
                            present_items[layout[row][col].items[lev][0]] += 1
                        else:
                            present_items[layout[row][col].items[lev][0]] = 1
        for item in layout.get_item_list():
            if item not in present_items.keys():
                missing_items.append(item)
        return present_items, missing_items
    
    for row in range(new_layout.shape[0]):
        for col in range(new_layout.shape[1]):
            if new_layout[row][col].type.name == 'RACK':
                for lev in range(new_layout.get_max_rack_level()):
                    present_items, missing_items_array = get_present_and_missing_items(new_layout)
                    most_popular = sorted(present_items.items(), key=lambda x: x[1], reverse=True)[:15]
                    most_popular = [x[0] for x in most_popular]
                    if len(most_popular) == 15 and len(missing_items_array) > 0:
                        # increased chance of mutation for popular items
                        if random.random() < alpha*2 and new_layout[row][col].items[lev][0] in most_popular:
                            item = random.choice(missing_items_array)
                            new_layout.set_item_to_rack(item, (row, col), level=lev)
                    else:
                        if random.random() < alpha:
                            item = random.choice(new_layout.get_item_list())
                            new_layout.set_item_to_rack(item, (row, col), level=lev)
                        
    return new_layout

In [178]:
def mutate_uniformity_for_rack(original_layout, alpha=0.01):
    new_layout = original_layout.copy()
    new_layout.reset_path_count()
    for row in range(new_layout.shape[0]):
        for col in range(new_layout.shape[1]):
            if new_layout[row][col].type.name == 'RACK':
                for lev in range(new_layout.get_max_rack_level()):
                    if random.random() < alpha:
                        items = [x[0] for x in new_layout[row][col].items]
                        if len(items) > 1:
                            items.pop(lev)
                        new_layout.set_item_to_rack(random.choice(items), (row, col), level=lev)
    return new_layout

In [179]:
temp = mutate_layout(new_layouts[0], alpha=0.1)

In [180]:
dir = Path(os.getcwd()).parent
temp.display_in_window(home_dir=str(dir))

# Genetic algorithm

In [181]:
layouts_array = []
for i in range(50):
    layouts_array.append(layout_with_one_item())

In [182]:
# evaluate, select, mutate, repeat
def genetic_algorithm(
        layouts_arr, 
        check_arr, 
        n_iter=100, 
        n_best=10, 
        alpha=0.01, 
        start_pos=None, 
        debug=False, 
        reward_type='max', 
        use_item_count=False, 
        weights=(500, 150, 350),
        rule_based_mutations=False,
        alpha_change_func: callable = None):
    history = []
    l_size = len(layouts_arr)
    check_arr.sort(key=lambda x: len(x[1]))
    mutation_type = ""
    for i in trange(n_iter):
        if debug:
            print(f'Iteration {i} \'{mutation_type}\'', end=' ')
        if alpha_change_func:
            alpha = alpha_change_func(i, n_iter, alpha)
        n_best_arr, hi, res_dict = select_n_best_layouts(layouts_arr, check_arr, n_best=n_best, start_pos=start_pos, debug=debug, reward_type=reward_type, use_item_count=use_item_count, weights=weights)
        history.append(hi)
        layouts_arr = []
        for l in n_best_arr:
            layouts_arr.append(l.copy().reset_path_count())
        for n in range(len(n_best_arr)):
            for m in range(l_size//n_best-1):
                if rule_based_mutations:
                    if random.random() < 0.32:
                        layouts_arr.append(mutate_uniformity(n_best_arr[n], alpha=alpha))
                        mutation_type = "LU" # layout uniformity
                    elif random.random() < 0.64:
                        layouts_arr.append(mutate_missing_items(n_best_arr[n], list(flatten(res_dict[random.randint(0, n_best-1)])), alpha=alpha))
                        mutation_type = "MI" # missing items
                    elif random.random() < 0.96:
                        layouts_arr.append(mutate_uniformity_for_rack(n_best_arr[n], alpha=alpha))
                        mutation_type = "RU" # rack uniformity
                    else:
                        layouts_arr.append(mutate_layout(n_best_arr[n], alpha=alpha))
                        mutation_type = "RI" # random item
                else:
                    layouts_arr.append(mutate_layout(n_best_arr[n], alpha=alpha))
    print(f'Iteration {n_iter} \'{mutation_type}\'', end=' ')
    res, hi, res_dict = select_n_best_layouts(layouts_arr, check_arr, n_best=n_best, start_pos=start_pos, debug=debug, reward_type=reward_type, use_item_count=use_item_count, weights=weights)
    history.append(hi)
    return res, history

In [193]:
score_best_layouts, score_history = genetic_algorithm(layouts_array, check_list[:400], n_iter=300, n_best=2, alpha=0.005, debug=True, reward_type='score', use_item_count=True, weights=(700, 150, 150))

  0%|          | 0/300 [00:00<?, ?it/s]

Iteration 0 '' Scores: [735.75, 735.75]
Iteration 1 '' Scores: [735.4150943396226, 735.75]
Iteration 2 '' Scores: [727.0377358490566, 735.4150943396226]
Iteration 3 '' Scores: [727.0377358490566, 727.0377358490566]
Iteration 4 '' Scores: [725.3254716981132, 727.0377358490566]
Iteration 5 '' Scores: [724.6556603773585, 725.3254716981132]
Iteration 6 '' Scores: [721.9009433962265, 721.9009433962265]
Iteration 7 '' Scores: [714.566037735849, 715.2735849056603]
Iteration 8 '' Scores: [713.8962264150944, 713.8962264150944]
Iteration 9 '' Scores: [709.3915094339623, 712.5188679245282]
Iteration 10 '' Scores: [706.5990566037736, 709.3915094339623]
Iteration 11 '' Scores: [701.7216981132075, 705.2594339622641]
Iteration 12 '' Scores: [700.382075471698, 701.0330188679245]
Iteration 13 '' Scores: [698.2971698113208, 699.6556603773585]
Iteration 14 '' Scores: [694.1273584905659, 698.2971698113208]
Iteration 15 '' Scores: [692.4150943396226, 693.7924528301887]
Iteration 16 '' Scores: [692.41509433

In [194]:
curr_dir = Path(os.getcwd()).parent

In [256]:
score_best_layouts[0].display_in_window(home_dir=str(curr_dir), debug=True)

[pywebview] Using GTK


In [196]:
print(score_best_layouts[1].get_layout_json())

{"hideSaveLoadButtons":false,"rackLevels":4,"items":["baking ingredients","soy lactosefree","butter","fresh vegetables","yogurt","canned meals beans","poultry counter","ice cream ice","fresh fruits","milk","packaged cheese","bread","tea","bakery desserts","frozen breakfast","cereal","eggs","buns rolls","cream","water seltzer sparkling water","pickled goods olives","packaged poultry","other creams cheeses","honeys syrups nectars","coffee","refrigerated","energy granola bars","soft drinks","plates bowls cups flatware","paper goods","food storage","nuts seeds dried fruit","packaged vegetables fruits","hot dogs bacon sausage","lunch meat","chips pretzels","meat counter","fresh dips tapenades","prepared soups salads","condiments","juice nectars","canned fruit applesauce","preserved dips spreads","packaged produce","canned jarred vegetables","fresh pasta","pasta sauce","frozen produce","frozen appetizers sides","soup broth bouillon","dry pasta","prepared meals","fresh herbs","hot cereal panc

In [197]:
def save_layouts(layouts_to_save, metric):
    for i in range(len(layouts_to_save)):
        filename = f'./../data/layouts/genetic/{metric}/layout_{i}.json'
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename, 'w') as f:
            f.write(layouts_to_save[i].get_layout_json())

In [198]:
save_layouts(score_best_layouts, 'score')

In [247]:
def plot_score_history(history):
    fig = px.line(x=range(len(history)), y=[x[0] for x in history], title='Score', 
                  labels={'value': 'Score', 'x': 'Iteration'}, height=700, width=700)
    fig.show()

In [248]:
plot_score_history(score_history)

## max metric

In [201]:
max_best_layouts, max_history = genetic_algorithm(layouts_array, check_list[:400], n_iter=300, n_best=2, alpha=0.005, debug=True, reward_type='max', use_item_count=True)

  0%|          | 0/300 [00:00<?, ?it/s]

Iteration 0 '' ['P:18, I:399, U:0,53', 'P:18, I:399, U:0,53']
Iteration 1 '' ['P:110, I:396, U:16,57', 'P:58, I:397, U:12,56']
Iteration 2 '' ['P:303, I:387, U:22,59', 'P:263, I:389, U:20,58']
Iteration 3 '' ['P:416, I:383, U:36,63', 'P:371, I:386, U:22,59']
Iteration 4 '' ['P:529, I:379, U:44,65', 'P:497, I:380, U:40,64']
Iteration 5 '' ['P:605, I:377, U:44,66', 'P:591, I:377, U:57,70']
Iteration 6 '' ['P:692, I:374, U:61,72', 'P:677, I:375, U:50,68']
Iteration 7 '' ['P:744, I:373, U:69,74', 'P:734, I:373, U:63,73']
Iteration 8 '' ['P:852, I:370, U:71,75', 'P:813, I:371, U:71,75']
Iteration 9 '' ['P:938, I:368, U:77,77', 'P:908, I:369, U:75,76']
Iteration 10 '' ['P:1026, I:365, U:85,79', 'P:1016, I:366, U:83,79']
Iteration 11 '' ['P:1114, I:363, U:85,80', 'P:1069, I:364, U:93,81']
Iteration 12 '' ['P:1175, I:361, U:97,82', 'P:1159, I:362, U:89,81']
Iteration 13 '' ['P:1254, I:360, U:97,83', 'P:1212, I:360, U:101,83']
Iteration 14 '' ['P:1512, I:352, U:107,85', 'P:1298, I:359, U:101,85

In [202]:
save_layouts(max_best_layouts, 'max')

In [249]:
def plot_history(history, mode='max'):
    path = [x[0]['path'] for x in history]
    if mode == 'min':
        path = [x[0]['path'] - x[0]['invalid']*1000 for x in history]
    invalid = [x[0]['invalid'] for x in history]
    print(len(path), len(invalid))
    fig = px.line(x=range(len(history)), y=[path, invalid], title='Path count', 
                  labels={'value': 'Count', 'x': 'Iteration'}, height=700, width=700)
    new_names = {'wide_variable_0': 'Path', 'wide_variable_1': 'Invalid'}
    fig.for_each_trace(lambda t: t.update(name=new_names[t.name]))
    fig.show()

In [250]:
plot_history(max_history)

301 301


In [251]:
max_best_layouts[0].display_in_window(home_dir=str(curr_dir), debug=True)

[pywebview] Using GTK


## min metric

In [206]:
min_best_layouts, min_history = genetic_algorithm(layouts_array, check_list[:400], n_iter=300, n_best=2, alpha=0.005, debug=True, reward_type='min', use_item_count=True)

  0%|          | 0/300 [00:00<?, ?it/s]

Iteration 0 '' ['P:399018, I:399, U:0,53', 'P:399018, I:399, U:0,53']
Iteration 1 '' ['P:396086, I:396, U:12,56', 'P:397070, I:397, U:10,56']
Iteration 2 '' ['P:394150, I:394, U:18,58', 'P:394166, I:394, U:22,60']
Iteration 3 '' ['P:391228, I:391, U:26,60', 'P:391257, I:391, U:28,62']
Iteration 4 '' ['P:390244, I:390, U:31,62', 'P:390244, I:390, U:27,61']
Iteration 5 '' ['P:387345, I:387, U:33,63', 'P:388284, I:388, U:35,63']
Iteration 6 '' ['P:380530, I:380, U:41,65', 'P:381511, I:381, U:39,64']
Iteration 7 '' ['P:378598, I:378, U:53,68', 'P:379550, I:379, U:49,67']
Iteration 8 '' ['P:375704, I:375, U:60,71', 'P:377614, I:377, U:54,69']
Iteration 9 '' ['P:370825, I:370, U:62,72', 'P:374713, I:374, U:68,73']
Iteration 10 '' ['P:367946, I:367, U:72,75', 'P:369861, I:369, U:66,73']
Iteration 11 '' ['P:366037, I:365, U:80,77', 'P:366982, I:366, U:74,76']
Iteration 12 '' ['P:365054, I:364, U:78,77', 'P:365057, I:364, U:84,78']
Iteration 13 '' ['P:362143, I:361, U:88,80', 'P:362163, I:361, 

In [207]:
save_layouts(min_best_layouts, 'min')

In [252]:
plot_history(min_history, 'min')

301 301


In [253]:
min_best_layouts[0].display_in_window(home_dir=str(curr_dir), debug=True)

[pywebview] Using GTK


# Simulated Annealing

In [210]:
def get_checks_of_specific_length(check_list, length):
    return [x for x in check_list if len(x[1]) == length]

def get_checks_of_specific_length_range(check_list, range_dict):
    # range dict: length - n_of_checks
    res = []
    for key in tqdm(range_dict.keys()):
        res += get_checks_of_specific_length(check_list, key)[:range_dict[key]]
    return res

check_config = {
    1: 30,
    2: 50,
    3: 60,
    4: 75,
    5: 75,
    6: 60,
    7: 50,
}

tuned_checks = get_checks_of_specific_length_range(check_list, check_config)


  0%|          | 0/7 [00:00<?, ?it/s]

In [211]:
def get_temperature(tested_layout, test_check_list, return_layout=False):
    _layout = tested_layout.copy()
    res, proc_layout = thread_func(_layout, test_check_list, use_item_count=True)
    if return_layout:
        return 1000 - calculate_score(res, proc_layout, test_check_list, (600, 200, 200)), proc_layout
    return 1000 - calculate_score(res, proc_layout, test_check_list, (600, 200, 200)), res

In [212]:
def simulated_annealing(initial_layout, initial_temperature, cooling_rate, checks, max_iter = 10000, prob=0.5, step=1, rule_based_mutations=False, alpha=0.05):
    current_layout = initial_layout
    temperature = initial_temperature
    history = {}
    res = {}
    _best_layout = current_layout
    best_temperature = 0
    for i in trange(max_iter):
        if temperature > 1:
            #new_layout = mutate_layout(current_layout, alpha=0.05)
            if rule_based_mutations:
                if random.random() < 0.32:
                    new_layout = mutate_uniformity(current_layout, alpha=alpha)
                elif random.random() < 0.64:
                    new_layout = mutate_missing_items(current_layout, list(flatten(res['missing_items'])), alpha=alpha)
                elif random.random() < 0.96:
                    new_layout = mutate_uniformity_for_rack(current_layout, alpha=alpha)
                else:
                    new_layout = mutate_layout(current_layout, alpha=alpha)
            else:
                new_layout = mutate_layout(current_layout, alpha=alpha)
            new_temperature, res = get_temperature(new_layout.copy(), checks)
            if new_temperature > temperature:
                history[i] = [new_temperature, 'higher']
                temperature = new_temperature
                current_layout = new_layout
                print(f'{i}: New layout accepted with temperature {new_temperature}')
            else:
                if random.random() < np.exp((new_temperature - temperature) / temperature - prob)**step:
                    history[i] = [new_temperature, 'lower']
                    current_layout = new_layout
                    print(f'{i}: New layout accepted with temperature {new_temperature}. Reason: random.')
            temperature *= (1 - cooling_rate)
        else:
            break
        if temperature > best_temperature:
            _best_layout = current_layout
            best_temperature = temperature
            print(f'\n*** New best layout found with temperature {best_temperature} ***\n')
    return current_layout, _best_layout, history

In [213]:
initial_layout = random_layout()

In [214]:
curr, best_layout, iter_history = simulated_annealing(initial_layout, 3, 0.01, tuned_checks, max_iter=4000, prob=0.7, alpha=0.15)

  0%|          | 0/4000 [00:00<?, ?it/s]

0: New layout accepted with temperature 245.07547169811323

*** New best layout found with temperature 242.6247169811321 ***
1: New layout accepted with temperature 208.13207547169804. Reason: random.
3: New layout accepted with temperature 204.7452830188679. Reason: random.
5: New layout accepted with temperature 200.63207547169816. Reason: random.
7: New layout accepted with temperature 206.63207547169816. Reason: random.
14: New layout accepted with temperature 193.68867924528297. Reason: random.
16: New layout accepted with temperature 214.13207547169816
17: New layout accepted with temperature 213.0188679245283
18: New layout accepted with temperature 234.18867924528297
19: New layout accepted with temperature 226.68867924528297. Reason: random.
20: New layout accepted with temperature 226.68867924528297. Reason: random.
22: New layout accepted with temperature 243.18867924528297
25: New layout accepted with temperature 193.68867924528297. Reason: random.
28: New layout accepted w

In [215]:
best_layout.display_in_window(home_dir=str(curr_dir), debug=True)

[pywebview] Using GTK


In [216]:
score, layout = get_temperature(best_layout, tuned_checks, return_layout=True)

In [217]:
score

287.63207547169804

In [218]:
layout.display_in_window(home_dir=str(curr_dir), debug=True)

[pywebview] Using GTK


In [219]:
def plot_temperature_history(history):
    # plot a line with red dots for higher, blue for lower
    fig = px.line(x=history.keys(), y=[x[0] for x in history.values()], title='Temperature', 
                  labels={'value': 'Temperature', 'x': 'Iteration'}, height=700)
    fig.add_scatter(x=[x for x in history.keys() if history[x][1] == 'higher'], y=[history[x][0] for x in history.keys() if history[x][1] == 'higher'], mode='markers', marker=dict(color='red'))
    fig.add_scatter(x=[x for x in history.keys() if history[x][1] == 'lower'], y=[history[x][0] for x in history.keys() if history[x][1] == 'lower'], mode='markers', marker=dict(color='blue'))
    # plot highest temperature as green dot
    sorted_dict = dict(sorted(history.items(), key=lambda item: item[1][0], reverse=True))
    fig.add_scatter(x=[list(sorted_dict.keys())[0]], y=[list(sorted_dict.values())[0][0]], mode='markers', marker=dict(color='green'))
    fig.show()

In [220]:
plot_temperature_history(iter_history)

In [225]:
curr_high_prob, best_layout_high_prob, iter_history = simulated_annealing(initial_layout, 3, 0.005, tuned_checks, max_iter=8000, prob=0.2, alpha=0.05, rule_based_mutations=True)

  0%|          | 0/8000 [00:00<?, ?it/s]

0: New layout accepted with temperature 205.17924528301887

*** New best layout found with temperature 204.1533490566038 ***
1: New layout accepted with temperature 213.2358490566038

*** New best layout found with temperature 212.16966981132077 ***
2: New layout accepted with temperature 229.73584905660368

*** New best layout found with temperature 228.58716981132068 ***
3: New layout accepted with temperature 230.67924528301876

*** New best layout found with temperature 229.52584905660368 ***
4: New layout accepted with temperature 236.0

*** New best layout found with temperature 234.82 ***
5: New layout accepted with temperature 238.87735849056617

*** New best layout found with temperature 237.68297169811333 ***
6: New layout accepted with temperature 245.43396226415098

*** New best layout found with temperature 244.20679245283023 ***
7: New layout accepted with temperature 238.4905660377359. Reason: random.
8: New layout accepted with temperature 249.16037735849068

*** New be

In [226]:
best_layout_high_prob.display_in_window(home_dir=str(curr_dir), debug=True)

[pywebview] Using GTK


In [227]:
plot_temperature_history(iter_history)

In [230]:
score, layout = get_temperature(best_layout_high_prob, tuned_checks, return_layout=True)

In [231]:
score

401.5471698113207

In [ ]:
layout.display_in_window(home_dir=str(curr_dir), debug=True)

# Combinations

In [232]:
# display diagram with check length
def plot_check_length(check_list):
    fig = px.histogram(x=[len(x[1]) for x in check_list], title='Check length', labels={'value': 'Length', 'x': 'Check'}, height=700)
    fig.show()

plot_check_length(check_list)

In [233]:
def get_checks_of_specific_length(check_list, length):
    return [x for x in check_list if len(x[1]) == length]

def get_checks_of_specific_length_range(check_list, range_dict):
    # range dict: length - n_of_checks
    res = []
    for key in tqdm(range_dict.keys()):
        res += get_checks_of_specific_length(check_list, key)[:range_dict[key]]
    return res

In [234]:
check_config = {
    1: 30,
    2: 50,
    3: 60,
    4: 75,
    5: 75,
    6: 60,
    7: 50,
}

In [235]:
tuned_checks = get_checks_of_specific_length_range(check_list, check_config)

layouts_array = []
for i in range(50):
    layouts_array.append(layout_with_one_item(4))

  0%|          | 0/7 [00:00<?, ?it/s]

In [236]:
def alpha_change_func(i, alpha, n_iter):
    new_alpha = alpha
    last_num = i // 10
    if last_num == 0:
        new_alpha = 0.8
    elif last_num == 1:
        new_alpha = 0.7
    elif last_num == 2:
        new_alpha = 0.6
    elif last_num == 3:
        new_alpha = 0.5
    elif last_num == 4:
        new_alpha = 0.4
    elif last_num == 5:
        new_alpha = 0.3
    elif last_num == 6:
        new_alpha = 0.2
    elif last_num == 7:
        new_alpha = 0.1
    elif last_num == 8:
        new_alpha = 0.05
    elif last_num == 9:
        new_alpha = 0.01
    return new_alpha

In [237]:
def alpha_change_func2(i, alpha, n_iter):
    new_alpha = alpha
    if i % 10 == 0:
        new_alpha = alpha - 0.1
    if new_alpha < 0.02:
        new_alpha = 0.02
    return new_alpha

In [239]:
step1_best_layouts, step1_history = genetic_algorithm(
    layouts_array, 
    tuned_checks, 
    n_iter=350, 
    n_best=2, 
    alpha=0.005, 
    debug=True, 
    reward_type='max', 
    use_item_count=True, 
    rule_based_mutations=True)

  0%|          | 0/350 [00:00<?, ?it/s]

Iteration 0 '' ['P:0, I:400, U:0,53', 'P:0, I:400, U:0,53']
Iteration 1 'MI' ['P:131, I:396, U:8,56', 'P:48, I:398, U:4,54']
Iteration 2 'RU' ['P:335, I:390, U:10,57', 'P:131, I:396, U:8,56']
Iteration 3 'MI' ['P:407, I:388, U:22,60', 'P:399, I:388, U:14,58']
Iteration 4 'MI' ['P:454, I:386, U:20,60', 'P:427, I:387, U:25,61']
Iteration 5 'MI' ['P:454, I:386, U:20,60', 'P:454, I:386, U:20,60']
Iteration 6 'MI' ['P:515, I:384, U:28,62', 'P:515, I:384, U:24,61']
Iteration 7 'LU' ['P:668, I:380, U:34,64', 'P:574, I:382, U:36,64']
Iteration 8 'MI' ['P:705, I:379, U:36,65', 'P:668, I:380, U:34,64']
Iteration 9 'MI' ['P:705, I:379, U:36,65', 'P:705, I:379, U:36,65']
Iteration 10 'MI' ['P:813, I:376, U:40,66', 'P:747, I:378, U:40,66']
Iteration 11 'MI' ['P:923, I:373, U:50,69', 'P:881, I:374, U:44,68']
Iteration 12 'RU' ['P:1081, I:368, U:62,72', 'P:988, I:371, U:52,70']
Iteration 13 'MI' ['P:1290, I:362, U:68,74', 'P:1089, I:368, U:58,72']
Iteration 14 'LU' ['P:1331, I:361, U:73,76', 'P:1329,

In [240]:
plot_history(step1_history)

351 351


In [241]:
save_layouts(step1_best_layouts, 'step1-mini-max')

In [242]:
valid_layouts_mutated = []
for best in step1_best_layouts:
    for i in range(50):
        valid_layouts_mutated.append(mutate_layout(best, alpha=0.01))

In [245]:
step2_best_layouts, step2_history = genetic_algorithm(valid_layouts_mutated, tuned_checks, n_iter=190, n_best=2, alpha=0.001, debug=True, reward_type='score', use_item_count=True, weights=(300, 200, 500), rule_based_mutations=True)

  0%|          | 0/190 [00:00<?, ?it/s]

Iteration 0 '' Scores: [558.5094339622642, 558.5094339622642]
Iteration 1 'MI' Scores: [558.5094339622642, 558.5094339622642]
Iteration 2 'LU' Scores: [556.1509433962265, 558.5094339622642]
Iteration 3 'MI' Scores: [553.7924528301887, 556.1509433962265]
Iteration 4 'MI' Scores: [553.7924528301887, 553.7924528301887]
Iteration 5 'MI' Scores: [551.9905660377359, 552.8254716981132]
Iteration 6 'RU' Scores: [551.9669811320755, 551.9905660377359]
Iteration 7 'RU' Scores: [549.6320754716982, 551.9669811320755]
Iteration 8 'MI' Scores: [549.0754716981132, 549.6084905660377]
Iteration 9 'LU' Scores: [549.0754716981132, 549.0754716981132]
Iteration 10 'MI' Scores: [548.1320754716982, 549.0754716981132]
Iteration 11 'RU' Scores: [547.4669811320755, 548.1320754716982]
Iteration 12 'LU' Scores: [547.3584905660377, 547.4669811320755]
Iteration 13 'MI' Scores: [544.9150943396227, 545.8584905660377]
Iteration 14 'LU' Scores: [544.25, 544.7216981132076]
Iteration 15 'RU' Scores: [543.3915094339623, 54

In [246]:
plot_score_history(step2_history)

In [ ]:
curr_dir = Path(os.getcwd()).parent
step2_best_layouts[0].display_in_window(home_dir=str(curr_dir), debug=True)

# merging layout items ito categories

idea - create a dictionary with categories as keys and items as values
based on this dictionary mark layout sells with categories (more items from category - category is marked)
if there are more items from different categories - do not mark

In [ ]:
categories = dict() # item - category
item_count = df['product_name'].unique().shape[0]

In [ ]:
products = df[['product_name', 'department']].drop_duplicates()

In [ ]:
for index, row in products.iterrows():
    categories[row['product_name']] = row['department']

In [ ]:
sorted(categories.items(), key=lambda x: x[1])

In [ ]:
def get_category_for_item(item):
    return categories[item]

def set_category_for_items(layout):
    def select_category(items):
        counts = dict()
        for item in items:
            if item in categories.keys():
                if categories[item] in counts.keys():
                    counts[item] += 1
                else:
                    counts[item] = 1
        if len(counts) == 4:
            return ""
        else:
            sorted(counts.items(), key=lambda x: x[1], reverse=True)
            return categories[list(counts.keys())[0]]
    
    for row in range(layout.shape[0]):
        for col in range(layout.shape[1]):
            if layout[row][col].type.name == 'RACK':
                layout[row][col].category = select_category([x[0] for x in layout[row][col].items])
                if layout[row][col].category:
                    print(f"Category: {layout[row][col].category} for {[x[0] for x in layout[row][col].items]}")
    return layout

In [ ]:
step2_best_layouts[0] = set_category_for_items(step2_best_layouts[0])

In [ ]:
step2_best_layouts[0].display_in_window(home_dir=str(curr_dir), debug=True)

In [ ]:
step2_best_layouts[0].get_layout_json()